In [12]:
import numpy as np
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d
from scipy.stats import norm # Import norm for BS formula

def crank_nicolson_american_put_log_price_fixed(S0, K, T, r, sigma, M, N, S_min=0.01, S_max=50, S_max_factor=2):
    """
    Calculates the American Put Option price using the Crank-Nicolson Finite Difference Method.
    Uses log-asset price (x = ln(S)) coordinates for numerical stability.
    Broadcast error in tridiagonal multiplication has been fixed.
    """

    # --- 1. Grid Setup and Step Sizes (x = ln(S) Coordinate) ---
    dt = T / M               # Time step size
    
    # Define X coordinate range [X_min, X_max]
    # Note: S_max_factor is used to set the upper boundary relative to K
    X_max = np.log(K * S_max_factor)
    X_min = np.log(S_min) 
    
    dX = (X_max - X_min) / N # Space step size
    
    # Asset price and time grid
    X = np.linspace(X_min, X_max, N + 1)
    S = np.exp(X) # Actual price grid
    
    V = np.zeros((N + 1, M + 1))

    # --- 2. Fixed Constant Coefficients (in X Coordinate) ---
    # These coefficients are constant because we are in the log-price coordinate
    # a * V_{i-1} + b * V_{i} + c * V_{i+1}
    a = 0.25 * dt * (sigma**2 / dX**2 - (r - 0.5 * sigma**2) / dX)
    b = 0.5 * dt * (sigma**2 / dX**2 + r)
    c = 0.25 * dt * (sigma**2 / dX**2 + (r - 0.5 * sigma**2) / dX)
    
    # --- 3. Construct Banded Storage for Tridiagonal Matrix M_L ---
    
    # M_L: Implicit part (Left Matrix)
    # M_L = diag(1 + b) + diag(-a, k=-1) + diag(-c, k=1)
    ML_bands = np.zeros((3, N - 1))
    ML_bands[0, 1:] = -c     # Upper diagonal (i+1)
    ML_bands[1, :] = 1 + b   # Main diagonal (i)
    ML_bands[2, :-1] = -a   # Lower diagonal (i-1)
    
    # --- 4. Boundary and Terminal Conditions ---
    V[:, M] = np.maximum(0, K - S) # Terminal condition
    time_points = np.linspace(0, T, M + 1)
    V[0, :] = K * np.exp(-r * (T - time_points)) # S=S_min Boundary (Discounted K)
    V[N, :] = 0 # S=S_max Boundary

    # --- 5. Backward Solving (Backward Induction) ---
    for j in range(M - 1, -1, -1):
        V_next = V[1:N, j + 1] # Length N-1 (known values at t_{j+1})
        
        # 5.1. Calculation of M_R * V^{j+1} (N-1 dimensional vector)
        MR_V_next = np.zeros(N - 1)
        
        # Main diagonal: (1 - b) * V_{i}^{j+1}
        MR_V_next += (1 - b) * V_next 
        
        # Upper diagonal: c * V_{i+1}^{j+1} (Applies to MR_V_next[0] through MR_V_next[N-3])
        MR_V_next[:-1] += c * V_next[1:] 
        
        # Lower diagonal: a * V_{i-1}^{j+1} (Applies to MR_V_next[1] through MR_V_next[N-2])
        MR_V_next[1:] += a * V_next[:-1] 

        # 5.2. Construct Right-Hand Side Vector R_Vector
        # R_Vector = M_R * V^{j+1} + Boundary_Terms_Explicit + Boundary_Terms_Implicit
        R_Vector = MR_V_next.copy()
        
        # Explicit boundary effect term (V^{j+1})
        # R_1 is affected by a * V_{0}^{j+1}
        R_Vector[0] += a * V[0, j + 1]             
        # R_{N-1} is affected by c * V_{N}^{j+1}
        R_Vector[-1] += c * V[N, j + 1]            

        # Implicit boundary effect term (V^{j})
        # R_1 is affected by -a * V_{0}^{j}, hence R_Vector[0] += -(-a * V[0, j])
        R_Vector[0] += a * V[0, j]                  
        # R_{N-1} is affected by -c * V_{N}^{j}, hence R_Vector[-1] += -(-c * V[N, j])
        R_Vector[-1] += c * V[N, j]                 
        
        # 5.3. Solve linear system M_L * V^{j} = R_Vector
        V_interior_Continuation = solve_banded((1, 1), ML_bands, R_Vector)
        
        # 5.4. Early exercise check (Projection)
        Intrinsic_Value = np.maximum(0, K - S[1:N])
        V[1:N, j] = np.maximum(Intrinsic_Value, V_interior_Continuation)

    # --- 6. Result Interpolation ---
    interpolator = interp1d(S, V[:, 0])
    
    if S0 < S_min or S0 > S_max:
         # S0 5.0, S_min 0.01, S_max 12.0. S0 is within range.
         # This is a safety check, unlikely to be triggered in normal execution
         print("Warning: S0 is outside the grid range defined by S_min and S_max_factor.")
         return "S0 is not within the grid range. Please adjust S_min or S_max_factor"

    price = interpolator(S0)
    
    return price

# --- Example Usage ---
S0 = 5.0
K = 6.0
T = 1.0
r = 0.05
sigma = 0.3
M = 1000  # Time steps
N = 1000  # Space steps (Price steps)

# Calculate American Put Option Price
P_American = crank_nicolson_american_put_log_price_fixed(S0, K, T, r, sigma, M, N)

# European Option Price Calculation (using analytical Black-Scholes formula)
def black_scholes_put(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    P_European = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    return P_European

P_European = black_scholes_put(S0, K, T, r, sigma)

# Calculate EEP
EEP = P_American - P_European

print(f"CN-FDM (M={M}, N={N}) American Put Option Price: {P_American:.7f}")
print(f"Black-Scholes European Put Option Price: {P_European:.7f}")
print(f"Early Exercise Premium (EEP): {EEP:.7f}")


CN-FDM (M=1000, N=1000) American Put Option Price: 1.1340355
Black-Scholes European Put Option Price: 1.0525764
Early Exercise Premium (EEP): 0.0814591
